In [1]:
import os
%cd ..



/home/jovyan


In [2]:
os.listdir()
os.listdir("data")


['diabetic_data.csv', 'IDs_mapping.csv']

In [3]:
!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.2/317.2 MB 1.7 MB/s eta 0:00:0000:0100:03
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 3.0 MB/s eta 0:00:00a 0:00:01
  Created wheel for pyspark: filename=pyspark-3.5.5-py2.py3-none-any.whl size=317747860 sha256=f03173a78dd1eca0cf1d9ddea8385b24368b1bc879ad2c05a1e7110d90b2230d
  Stored in directory: /home/jovyan/.cache/pip/wheels/0c/7f/b4/0e68c6d8d89d2e582e5498ad88616c16d7c19028680e9d3840
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession

# Start or get existing Spark session
spark = SparkSession.builder \
    .appName("HospitalReadmissionNotebook") \
    .getOrCreate()

# Check it's alive
spark



In [5]:
df= spark.read.csv("data/diabetic_data.csv", header= True, inferSchema = True) 
df.printSchema() 

root
 |-- encounter_id: integer (nullable = true)
 |-- patient_nbr: integer (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: string (nullable = true)
 |-- weight: string (nullable = true)
 |-- admission_type_id: integer (nullable = true)
 |-- discharge_disposition_id: integer (nullable = true)
 |-- admission_source_id: integer (nullable = true)
 |-- time_in_hospital: integer (nullable = true)
 |-- payer_code: string (nullable = true)
 |-- medical_specialty: string (nullable = true)
 |-- num_lab_procedures: integer (nullable = true)
 |-- num_procedures: integer (nullable = true)
 |-- num_medications: integer (nullable = true)
 |-- number_outpatient: integer (nullable = true)
 |-- number_emergency: integer (nullable = true)
 |-- number_inpatient: integer (nullable = true)
 |-- diag_1: string (nullable = true)
 |-- diag_2: string (nullable = true)
 |-- diag_3: string (nullable = true)
 |-- number_diagnoses: integer (nullable = true)
 |-

In [6]:
from pyspark.sql.functions import col,when,regexp_extract

#replace all '?' with null
df_clean= df.replace("?", None)


In [7]:
#extract numberic bin 
df_clean= df_clean.withColumn(
    "age_num", regexp_extract("age", r"\[(\d+)",1).cast("int") 
)

In [ ]:
#Readmitted Flag binary target 
df_clean = df_clean.withColumn(
    "readmitted_flag", when(col("readmitted") == "<30",1).otherwise(0)
)


In [8]:
#a loop to transform the medication into columns to simpler yes/no 

med_cols = ["insulin","metformin","glipizide","glyburide"]

for med in med_cols: 
    df_clean = df_clean.withColumn(f"{med}_used",
         when(col(med).isin("steady","up","Down"),1).otherwise(0)
                                 )

    
                                  



In [9]:
numeric_fields = [
    "time_in_hospital", "num_lab_procedures", "num_procedures",
    "num_medications", "number_outpatient", "number_emergency",
    "number_inpatient", "number_diagnoses"
]

for colname in numeric_fields:
    df_clean = df_clean.withColumn(colname, col(colname).cast("int"))


In [10]:
df_clean.write.mode("overwrite").parquet("output/patients_cleaned1.parquet")
